In [ ]:
import torch
from torch import nn
import tensorflow as tf
import numpy as np

In [ ]:
class TFModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.lin = tf.keras.layers.Dense(10, activation=tf.nn.relu)
    
    def call(self, inputs, training=False):
        return self.lin(inputs)

In [ ]:
class TorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(10, 10)
    
    def forward(self, x):
        return self.lin(x)

In [ ]:
tf_model = tf.function(TFModel())  
inp = np.random.random((10, 10))
x = tf.convert_to_tensor(inp, dtype=tf.float32)
tf_model(x)

tf_model.summary()

In [ ]:
torch_model = torch.compile(TorchModel())
inp2 = np.random.random((10, 10)).astype(np.float32)
x2 = torch.from_numpy(inp2)
torch_model, torch_model(x2).shape

### Creating the Profiler Logs TensorFlow

In [ ]:
logs = 'logs/' + "tensorflow"

from ivy.utils.profiler import tensorflow_profile_start, tensorflow_profile_stop
tensorflow_profile_start(logs, 
                         host_tracer_level = 3,
                         python_tracer_level = 1,
                         device_tracer_level = 1)
tf_model(x)
tensorflow_profile_stop()

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile 
!tensorboard --logdir='logs/'

### Creating the Profiler Logs Torch

In [ ]:
from ivy.utils.profiler import torch_profiler_init, torch_profiler_start, torch_profiler_stop
profiler = torch_profiler_init(activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA ],
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./logs/torch'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
torch_profiler_start(profiler)
torch_model(x2)
torch_profiler_stop(profiler)